In [60]:
import os
from pathlib import Path

if not 'workbook_dir' in globals():
    workbook_dir = os.getcwd()
print('workbook_dir: ' + workbook_dir)
os.chdir(Path(workbook_dir).parent.absolute())
print("current working dir:", os.getcwd())

workbook_dir: /Users/minor/projects/mcs-formalizations/queries
current working dir: /Users/minor/projects/mcs-formalizations


In [61]:
from rdflib import Graph, URIRef
from mcs_formalizations.namespace import bind_namespaces, MCS, RDF
from mcs_formalizations.helpers import uri_base

### Question: True/False - A parking garage is larger than an automobile.

##### Prepare the Graph

In [62]:
parking_garage = uri_base("parking_garage")
automobile = uri_base("automobile")
vehicle = uri_base("vehicle")

##### Minimum Knowledge Assumed:
- a parking garage contains vehicles
- an automobile is a type of vehicle

In [63]:
# TODO: move this generic process into a helper that takes the minimum knowledge
# The helper could be a class that wraps the graph e.g.,
# sparql_reasoner.add_knowledge(triples..)
# sparql_reasoner.add_rules(rule names)
# sparql_reasoner.ask(...)
g = Graph()
bind_namespaces(g)

g.add((parking_garage, MCS.contains, vehicle))
g.add((vehicle, RDF.type, automobile))
print("Graph after adding minimum knowledge:")
print(g.serialize(format="ttl").decode("utf-8"))

Graph after adding minimum knowledge:
@prefix mcs: <http://purl.org/twc/mcs/> .

<mcs:object:parking_garage> mcs:contains <mcs:object:vehicle> .

<mcs:object:vehicle> a <mcs:object:automobile> .




#### Minimum rules assumed:

- a container is larger than the things it contains

In [64]:
# TODO: move this process into a helper that takes the rule names
for rule_name in ("is_larger_than",):
    with open(Path("rules") / (rule_name + ".sparql"), "r") as sparql_file:
        sparql = sparql_file.read()
        for result in g.query(sparql):
            g.add(result)

print("Graph after adding rules:")
print(g.serialize(format="ttl").decode("utf-8"))

Graph after adding rules:
@prefix mcs: <http://purl.org/twc/mcs/> .

<mcs:object:parking_garage> mcs:contains <mcs:object:vehicle> ;
    mcs:isLargerThan <mcs:object:vehicle> .

<mcs:object:vehicle> a <mcs:object:automobile> .




In [65]:
qres_ask = g.query("ASK {mcs:object:parking_garage mcs:isLargerThan mcs:object:automobile}")
print(bool(qres_ask))

False
